In [1]:
from konlpy.tag import Komoran
import numpy as np

In [2]:
komoran = Komoran()
text = "오늘 날씨는 구름이 많아요."

# 명사만 추출
nouns = komoran.nouns(text)
print(nouns)

['오늘', '날씨', '구름']


In [3]:
# 단어 사전 구축 및 단어별 인덱스 부여
dics = {}
for word in nouns:
    if word not in dics.keys():
        dics[word] = len(dics)
print(dics)

{'오늘': 0, '날씨': 1, '구름': 2}


In [7]:
# 원-핫 인코딩
nb_classes = len(dics)
targets = list(dics.values())
one_hot_targets = np.eye(nb_classes)[targets]
print(one_hot_targets)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [10]:
# Word2Vec model 학습
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time

In [14]:
# 네이버 영화 리뷰 베이터 읽어옴
def read_review_data(filename):
    with open(filename, "r", encoding="utf-8") as f:
        data = [line.split("\t") for line in f.read().splitlines()]
        data = data[1:]
    return data

# 학습 시간 측정 시작
start = time.time()

# 리뷰 파일 읽어오기
print("1) 말뭉치 데이터 읽기 시작")
review_data = read_review_data("./ratings.txt")
print(len(review_data)) # 리뷰 데이터 전체 개수
print("1) 말뭉치 데이터 읽기 완료: ", time.time() - start)

# 문장 단위로 명사만 추출해서 학습 입력 데이터로 만듦
print("2) 형태소에서 명사만 추출 시작")
komoran = Komoran()
docs = [komoran.nouns(sentence[1]) for sentence in review_data]
print("2) 형태소에서 명사만 추출 완료: ", time.time() - start)

# Word2Vec 모델 학습
print("3) Word2Vec 모델 학습 시작")
model = Word2Vec(sentences=docs, vector_size=200, window=4, hs=1, min_count=2, sg=1)
print("3) Word2Vec 모델 학습 완료: ", time.time() - start)

# 모델 저장
print("4) 학습된 모델 저장 시작")
model.save("nvmc.model")
print("4) 학습된 모델 저장 완료: ", time.time() - start)

# 학습된 말뭉치 수, 코퍼스 내 전체 단어 수
print("corpus_count: ", model.corpus_count)
print("corpus_total_words: ", model.corpus_total_words)

1) 말뭉치 데이터 읽기 시작
200000
1) 말뭉치 데이터 읽기 완료:  1.9889864921569824
2) 형태소에서 명사만 추출 시작
2) 형태소에서 명사만 추출 완료:  89.36491060256958
3) Word2Vec 모델 학습 시작
3) Word2Vec 모델 학습 완료:  108.69314241409302
4) 학습된 모델 저장 시작
4) 학습된 모델 저장 완료:  109.22473001480103
corpus_count:  200000
corpus_total_words:  1076896


In [15]:
# Word2Vec model 활용
# 모델 불러오기
model = Word2Vec.load("nvmc.model")
print("corpus_total_words: ", model.corpus_total_words)

# "사랑"이란 단어로 생성한 단어 임베딩 벡터
print("사랑: ", model.wv["사랑"])

# 단어 유사도 계산
print("일요일 = 월요일\t", model.wv.similarity(w1="일요일", w2="월요일"))
print("안성기 = 배우\t", model.wv.similarity(w1="안성기", w2="배우"))
print("대기업 = 삼성\t", model.wv.similarity(w1="대기업", w2="삼성"))
print("일요일 != 삼성\t", model.wv.similarity(w1="일요일", w2="삼성"))
print("히어로 != 삼성\t", model.wv.similarity(w1="히어로", w2="삼성"))

corpus_total_words:  1076896
사랑:  [-0.03685663 -0.42167908  0.02685775 -0.14264534 -0.1700458  -0.3156721
 -0.01655827  0.3312561   0.2058018   0.1431381  -0.11901294 -0.15261705
 -0.0431291   0.08504339 -0.30478635  0.09490284 -0.06405049 -0.09885142
 -0.28437138 -0.26277325  0.1725525   0.14996408 -0.26925272  0.14353988
 -0.18371096  0.05570001  0.27569112 -0.05814886 -0.23663864 -0.0350089
  0.15608512  0.17034128 -0.0414695  -0.26766452  0.1963109   0.22288959
 -0.14951417 -0.127062   -0.0843439  -0.14024146 -0.18359353  0.19068392
 -0.04238747  0.08582901  0.12229525  0.24200384 -0.25764784 -0.02717914
  0.47539738  0.3758161   0.16036971 -0.03842648 -0.06018594 -0.13770464
  0.47064006 -0.0639108   0.3585746  -0.31894436 -0.1469196   0.02211447
 -0.11160678  0.02617247  0.04307424  0.11127619  0.13044347  0.02376742
 -0.5331304  -0.04104411  0.24096565  0.26641455  0.01180822 -0.09173312
  0.11698808 -0.03632953  0.01010222 -0.14943013  0.31790432 -0.14388713
 -0.39879224  0.122

In [17]:
# 가장 유사한 단어 추출
print(model.wv.most_similar("안성기", topn=5))
print(model.wv.most_similar("시리즈", topn=5))

[('씨야', 0.7206252217292786), ('정재영', 0.711334764957428), ('김갑수', 0.7031294703483582), ('장미희', 0.6973748803138733), ('고준희', 0.6943286061286926)]
[('나니아 연대기', 0.6817585825920105), ('캐리비안의 해적', 0.6745085120201111), ('더 울버린', 0.6623805165290833), ('X맨', 0.6457725763320923), ('기사단', 0.6443246006965637)]
